In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch()

In [2]:
es

<Elasticsearch([{}])>

In [3]:
doc = {
    'author': 'kimchy',
    'text': 'Elasticsearch: cool. bonsai cool.',
    'timestamp': datetime.now(),
}
res = es.index(index="test-index", doc_type='tweet', id=1, body=doc)
print(res['result'])

created


In [14]:
import pandas as pd
import numpy as np

In [8]:
from espandas import Espandas

In [43]:
df = (100 * pd.DataFrame(np.round(np.random.rand(100, 5), 2))).astype(int)
df.columns = ['A', 'B', 'C', 'D', 'E']
df['indexId'] = (df.index + 100).astype(str)

In [44]:
INDEX = 'foo_index'
TYPE = 'bar_type'
esp = Espandas()

In [45]:
esp.es_write(df, INDEX, TYPE)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/espandas/espandas.py:85: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  df = df.reindex_axis(sorted(df.columns), axis=1)


In [90]:
def pd_elastic(df):
    from espandas import Espandas
    df['indexId'] = (df.index).astype(str)
    INDEX = input('index name: ')
    TYPE = input('type name: ')
    esp = Espandas()
    esp.es_write(df, INDEX, TYPE)

In [24]:
df = pd.read_csv('f100.csv')
f100_t = df.head(15)
f100_t['indexId'] = (f100_t.index).astype(str)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
help(pd.core.indexes.range.RangeIndex.astype)

Help on function astype in module pandas.core.indexes.base:

astype(self, dtype, copy=True)
    Create an Index with values cast to dtypes. The class of a new Index
    is determined by dtype. When conversion is impossible, a ValueError
    exception is raised.
    
    Parameters
    ----------
    dtype : numpy dtype or pandas type
    copy : bool, default True
        By default, astype always returns a newly allocated object.
        If copy is set to False and internal requirements on dtype are
        satisfied, the original data is used to create a new Index
        or the original Index is returned.
    
        .. versionadded:: 0.19.0



In [25]:
INDEX = 'f100_t_index'
TYPE = 'f100_t_type'
esp = Espandas()
esp.es_write(f100_t, INDEX, TYPE)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/espandas/espandas.py:85: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  df = df.reindex_axis(sorted(df.columns), axis=1)


In [23]:
help(esp.es_write)

Help on method es_write in module espandas.espandas:

es_write(df, index, doc_type, index_name='indexId') method of espandas.espandas.Espandas instance
    Insert a Pandas DataFrame into ElasticSearch
    :param df: the DataFrame, must contain the column 'indexId' for a unique identifier
    :param index: the ElasticSearch index
    :param doc_type: the ElasticSearch doc_type



In [29]:
type(f100_t.index)

pandas.core.indexes.range.RangeIndex

In [46]:
df = pd.read_csv('rsi_t.csv')

In [84]:
df.head()

,es_date,raw_date,btc_volume,btc_price,RSI_6,RSI_9,RSI_15,indexId
0,2010-07-24T00:00:00Z,20100724,10400.000000,0.053,NaN,NaN,NaN,0
1,2011-03-01T00:00:00Z,20110301,79956.000000,0.930,NaN,NaN,NaN,1
2,2011-06-04T00:00:00Z,20110604,0.001000,17.830,NaN,NaN,NaN,2
3,2011-08-08T00:00:00Z,20110808,10770.515208,8.890,NaN,NaN,NaN,3
4,2012-06-22T00:00:00Z,20120622,0.031337,6.490,NaN,NaN,NaN,4


In [81]:
df['indexId'] = (df.index).astype(str)
INDEX = 'rsi_index'
TYPE = 'rsi_type'
esp = Espandas()
esp.es_write(df, INDEX, TYPE)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/espandas/espandas.py:85: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  df = df.reindex_axis(sorted(df.columns), axis=1)
